In [23]:
import pandas as pd
from pandasql import sqldf

In [24]:
evanston = pd.read_csv('ev311.csv')

In [25]:
evanston.head()

,id,priority,source,category,date_created,date_completed,street,house_num,zip,description
0,1340563,NONE,gov.publicstuff.com,Fire Prevention - Inspection of a Commercial P...,2016-01-13T15:03:18Z,2016-01-19T16:51:26Z,Sheridan Road,606-612,60202.0,Please contact Debbie at Ext. 222
1,1826017,MEDIUM,Iframe,Water Service - Question or Concern,2016-08-12T14:35:12Z,2016-08-27T07:00:27Z,Washington St,930,NaN,Last spring we called you to report that our s...
2,1849204,MEDIUM,Iframe,Trees-Fallen limb or tree,2016-08-22T09:07:45Z,2016-08-24T07:05:32Z,Lincoln St,1183-1223,NaN,This isn't about a fallen tree or tree limb bu...
3,1880254,MEDIUM,iOS,Ask A Question / Send A Message,2016-09-01T09:03:54Z,2016-09-01T16:52:40Z,Callan Ave,1–111,60202.0,"Actually on back side of 621 Howard, growth in..."
4,1972582,MEDIUM,Iframe,Dead Animal on Public Property,2016-09-19T01:46:41Z,2016-09-27T11:28:50Z,Crain St,1524,NaN,This is not public property but I need help. ...


In [26]:
all_col = """select count(*) as total_observation,
count(category) as category_count 
from evanston
"""
sqldf(all_col)


,total_observation,category_count
0,36431,36431


In [49]:
#-- Find values of zip that appear in at least 100 rows
#-- Also get the count of each value

zip_100 ="""
select zip, count(*)
from evanston
group by zip
having count(*) >=100
"""


In [50]:
sqldf(zip_100)

,zip,count(*)
0,NaN,5528
1,60201.0,19054
2,60202.0,11165
3,60208.0,255


In [62]:
#-- Find the 5 most common values of street and the count of each
street_max_5 ="""
select street,count(*)
from evanston
where street is not null
group by street
order by count(*) desc
limit 5
"""

In [63]:
sqldf(street_max_5)

,street,count(*)
0,Chicago Avenue,1440
1,Sherman Avenue,1276
2,Central Street,1211
3,Davis Street,1154
4,Church Street,1114


In [71]:
street_trim="""
SELECT distinct street,
       -- Trim off unwanted characters from street
       trim(street, '0123456789 #/. th') AS cleaned_street
       from evanston
       order by street
       limit 10
"""

In [72]:
sqldf(street_trim)

,street,cleaned_street
0,None,None
1,1/2 Chicago Ave,Chicago Ave
2,1047B Chicago Ave,B Chicago Ave
3,13th Street,Stree
4,141A Callan Ave,A Callan Ave
5,141b Callan Ave,b Callan Ave
6,1624B Central St,B Central S
7,217A Dodge Ave,A Dodge Ave
8,221c Dodge Ave,c Dodge Ave
9,300c Dodge Ave,c Dodge Ave


In [98]:
# Count rows
count_row = """
select description from evanston
where description like '%trash%'
or description like '%garbage%'
"""

In [99]:
sqldf(count_row)

,description
0,Large hole in base due to animals. They are n...
1,3 trash carts need to be repaired/replaced.
2,Caller is requesting for his 2 compost bins be...
3,Trash at 2000 Pratt Ct
4,these items were just dumped in my drive way. ...
...,...
2546,We put our Christmas tree on the parkway last ...
2547,"Garbage overflow at Walgreens, 635 Chicago. Al..."
2548,Please clean up the debris and garbage that ha...
2549,The apartment building at 1305-1321 Oak has ov...


In [107]:
# category and description
cat_desc = """
select count(*) as  cat_description from evanston
where (description like '%trash%'
or description like '%garbage%') and
category not like '%trash%' and
category not like '%garbage%'
group by category
order by count(*) desc 
limit 5
"""

In [108]:
sqldf(cat_desc)

,cat_description
0,273
1,77
2,28
3,16
4,15


In [109]:
evanston.head(3)

,id,priority,source,category,date_created,date_completed,street,house_num,zip,description
0,1340563,NONE,gov.publicstuff.com,Fire Prevention - Inspection of a Commercial P...,2016-01-13T15:03:18Z,2016-01-19T16:51:26Z,Sheridan Road,606-612,60202.0,Please contact Debbie at Ext. 222
1,1826017,MEDIUM,Iframe,Water Service - Question or Concern,2016-08-12T14:35:12Z,2016-08-27T07:00:27Z,Washington St,930,NaN,Last spring we called you to report that our s...
2,1849204,MEDIUM,Iframe,Trees-Fallen limb or tree,2016-08-22T09:07:45Z,2016-08-24T07:05:32Z,Lincoln St,1183-1223,NaN,This isn't about a fallen tree or tree limb bu...


In [132]:
# - Concatenate house_num, a space, and street
# -- and trim spaces from the start of the results
house_num = """
select trim(house_num||' '||street) as address
from evanston
limit 5
"""

In [133]:
sqldf(house_num)

,address
0,606-612 Sheridan Road
1,930 Washington St
2,1183-1223 Lincoln St
3,1–111 Callan Ave
4,1524 Crain St


In [140]:
# -- Select the first word of the street value
street_num ="""
select split_part(street,' ', 1) as street_name,count(*)
from evanston
group by street_name
"""

In [141]:
sqldf(street_num)

PandaSQLException: (sqlite3.OperationalError) no such function: split_part
[SQL: 
select split_part(street,' ', 1) as street_name,count(*)
from evanston
group by street_name
]
(Background on this error at: https://sqlalche.me/e/14/e3q8)